# Lista de Exercícios 2

### Esta é a lista de exercícios práticos onde serão trabalhados conceitos apresentados e exercitados em sala de aula

### Aspectos como clareza e organização do código, comentários, raciocínio lógico, coerência no nome de variáveis, classes e funções, utilização das estruturas mais adequadas para a solução dos problemas e o funcionamento adequado serão analisados e irão compor o valor da nota final

### Procure entender exatamente o que se pede antes de iniciar a codificação. Quando se sabe o que é necessário fazer a codificação fica mais fácil, mais eficiente e mais divertida ;-)

### Faça o upload do notebook Jupyter completo nomeado da seguinte forma: nome_sobrenome_RA.ipynb. Arquivos com nomes incorretos serão desconsiderados

### Por fim, boa sorte e enjoy!

<hr>

## Exercício 1

### Considere um arquivo de entrada no formato CSV (comma separated values) com informações relativas a acidentes na aviação civil brasileira nos últimos 10 anos (arquivo anv.csv)

### As informações estão separadas pelo caracter separador ~ e entre “” (aspas) conforme o exemplo abaixo:

```javascript
"201106142171203"~"PPGXE"~"AEROCLUBE"~"AVIÃO"~"NEIVA INDUSTRIA AERONAUTICA"~"56- C"~"PAUL"~"PISTÃO"~"MONOMOTOR"~"660"~"LEVE"~"2"~"1962"~"BRASIL"~"BRASIL"~"PRI"~" INSTRUÇÃO"~"SDPW"~"SDPW"~"INDETERMINADA"~"UNKNOWN"~"VOO DE INSTRUÇÃO"~"SUBSTANCIAL"~"0"~"2018-07-09“

"201707111402595"~"PPNCG"~"OPERADOR PARTICULAR"~"AVIÃO"~"PIPER AIRCRAFT"~"PA-46- 350P"~"PA46"~"TURBOÉLICE"~"MONOMOTOR"~"1950"~"LEVE"~"6"~"1990"~"NULL"~"BRASIL"~" TPP"~"PARTICULAR"~"SBBR"~"SBGR"~"POUSO"~"LANDING"~"VOO PRIVADO"~"NENHUM"~"0"~"2018-07-09"'''
```

### O arquivo é composto das seguintes colunas:
1. codigo_ocorrencia
2. aeronave_matricula
3. aeronave_operador_categoria 
4. aeronave_tipo_veiculo
5. aeronave_fabricante
6. aeronave_modelo
7. aeronave_tipo_icao
8. aeronave_motor_tipo
9. aeronave_motor_quantidade 
10. aeronave_pmd 
11. aeronave_pmd_categoria 
12. aeronave_assentos 
13. aeronave_ano_fabricacao
14. aeronave_pais_fabricante 
15. aeronave_pais_registro 
16. aeronave_registro_categoria 
17. aeronave_registro_segmento 
18. aeronave_voo_origem 
19. aeronave_voo_destino 
20. aeronave_fase_operacao 
21. aeronave_fase_operacao_icao 
22. aeronave_tipo_operacao 
23. aeronave_nivel_dano 
24. total_fatalidades
25. aeronave_dia_extracao

### Crie uma função que efetue a leitura do arquivo e processe seus dados identificando o total de ocorrencias no período avaliado e, baseado nessa informação criar um arquivo de saída em formato JSON (statistics.json) com as seguintes as estatísticas (número de ocorrências e percentual do total) por fase de operação (coluna aeronave_fase_operacao), como o exemplo a seguir:

```json

[
  {
    "fase_operacao": "APROXIMAÇÃO FINAL", 
    "ocorrencias": 234,
    "percentual": "4,51%"
  },
  {
    "fase_operacao": "INDETERMINADA", 
    "ocorrencias": 180,
    "percentual": "2,43%"
  },
  {
    "fase_operacao": "MANOBRA", 
    "ocorrencias": 80,
    "percentual": "0,95%"
  }
]

```

### Crie um arquivo de saida (formato CSV) com nome levels.csv contendo as seguintes informações:
* operation -> aeronave_operador_categoria
* type -> aeronave_tipo_veiculo
* manufacturer -> aeronave_fabricante
* engine_type aeronave_motor_tipo
* engines -> aeronave_motor_quantidade
* year_manufacturing -> aeronave_ano_fabricacao 
* seating -> aeronave_assentos
* fatalities -> total_fatalidades

### Considerando apenas acidentes cujo nível de dano da aeronave tenha sido LEVE ou NENHUM (coluna aeronave_nivel_dano) E que o número de fatalidades (total_fatalidades) tenha sido superior à 0 (zero)
OBS: 
1. Efetuar apenas uma leitura do arquivo de entrada
2. Efetuar todas as leituras e gravações de arquivos na pasta /tmp


In [177]:
import csv, json, tempfile

with tempfile.TemporaryDirectory() as tmpdirname:

    def air_stats(file):

        total_ocorrencias = 0 # sum the total of events
        d = {} # dict to store events as keys and count as values
        operacao = [] # list to store all dict keys and values

        # open file with csv library on tmp file
        with open(file, 'r', encoding="utf-8") as csv_file:
            csv_reader = csv.DictReader(csv_file, delimiter = '~')

            # iterate through source file
            for row in csv_reader:

                # count the total number of events
                total_ocorrencias += 1
                # store the dict key into a variable
                key = row['aeronave_fase_operacao']

                # check if event is stored into dict
                if key in d:
                    d[key] += 1
                else:
                    d[key] = 1

                # check conditions and write on list of events from dict
                if (row['aeronave_nivel_dano'] == 'LEVE' or row['aeronave_nivel_dano'] == 'NENHUM') and int(row['total_fatalidades']) > 0:
                    operacao.append([row['aeronave_operador_categoria'],
                                    row['aeronave_tipo_veiculo'],
                                    row['aeronave_fabricante'],
                                    row['aeronave_motor_tipo'],
                                    row['aeronave_motor_quantidade'],
                                    row['aeronave_ano_fabricacao'],
                                    row['aeronave_assentos'],
                                    row['total_fatalidades']])


        # ----------------------------
        # write JSON file        

        data = {} # new dict to store calculations and write
        data['estatisticas'] = []

        # iterate through dict 
        for key in d.keys():

            # perform calculations needed
            data['estatisticas'].append({
                'fase_operacao': key,
                'ocorrencias': d[key],
                'percentual': '{:.2f}%'.format((d[key]/total_ocorrencias)*100)
            })

        # write to JSON file forcing indent and encoding
        with open('statistics.json', 'w', encoding='utf-8') as outfile:
            json.dump(data['estatisticas'], outfile, ensure_ascii=False, indent=4)

        # ----------------------------
        # write CSV file

        # create header row
        header = ['operation',
                'type',
                'manufacturer',
                'engine_type',
                'engines',
                'year_manufacturing',
                'seating',
                'fatalities']

        # write to CSV file forcing encoding. Adding the header
        with open('levels.csv', 'w', encoding='utf-8') as csvFile:
            writer = csv.writer(csvFile)
            writer.writerow(header)
            for row in operacao:
                writer.writerow(row)

In [178]:
air_stats('anv.csv')

## Exercício 2

### Crie um programa que efetue a leitura do arquivo produzido no exercício 1 (levels.csv) e mostre na tela as seguintes informações estatísticas:
* Quantidade total de acidentes
* Quantidade total de acidentes agrupados por tipo de aeronave (type)
* Ano e fabricante da aeronave mais antiga (year_manufacturing, manufacturer)
* Quantidade de assentos e tipo de motor da aeronave mais nova (seating, engine_type)
* A quantidade de acidentes que ocorreram com aeronaves do tipo (type) HELICÓPTERO, cuja fabricação se deu após o ano 1997
* A quantidade de acidentes que ocorreram com aeronaves do tipo (type) AVIÃO fabricadas (manufacturer) pela CESSNA AIRCRAFT

OBS: 
1. Efetuar todas as leituras e gravações de arquivos na pasta /tmp

In [169]:
import csv

def levels_stats(file):
    
    total_acidentes = 0
    acidentes_aero = {}
    acidentes_heli = 0
    acidentes_aviao = 0
    mais_antiga = 9999
    mais_nova = 0
    
    with open(file, 'r', encoding="utf-8") as csv_file:
        csv_reader = csv.DictReader(csv_file) 
        
        for row in csv_reader:
            
            # count the total of events
            total_acidentes += 1
            
            # count the events per type
            key = row['type']
            if key in acidentes_aero:
                acidentes_aero[key] += 1
            else:
                acidentes_aero[key] = 1
                
            # search for oldest and store the year and manufacturer
            if int(row['year_manufacturing']) < mais_antiga:
                mais_antiga = int(row['year_manufacturing'])
                fabricante = row['manufacturer']
                
            # search for newest and store the year, seating and engine type
            if int(row['year_manufacturing']) > mais_nova:
                mais_nova = int(row['year_manufacturing'])
                assentos = row['seating']
                tipo_motor = row['engine_type']
                
            # count heli accidents after 1997
            if row['type'] == 'HELICÓPTERO' and int(row['year_manufacturing']) > 1997:
                acidentes_heli += 1
                
            # count plane accidents by CESSNA AIRCRAFT
            if row['type'] == 'AVIÃO' and row['manufacturer'] == 'CESSNA AIRCRAFT':
                acidentes_aviao += 1
                
        # --------------------------
        # print stats
        
        print('Quantidade total de acidentes: {}'.format(total_acidentes))
        for k, v in acidentes_aero.items():
            print('- tipo: {:12} acidentes: {:0}'.format(k,v))
        print('aeronave mais antiga: {} - fabricante: {}'.format(mais_antiga, fabricante))
        print('aeronave mais nova: {} - quantidade de assentos: {} - tipo de motor: {}'.format(mais_nova, assentos, tipo_motor))
        print('quantidade de acidentes de helicóptero após 1997: {}'.format(acidentes_heli))
        print('quantidade de acidentes de avião do fabricante CESSNA AIRCRAFT: {}'.format(acidentes_aviao))                

In [170]:
levels_stats('levels.csv')

Quantidade total de acidentes: 13
- tipo: AVIÃO        acidentes: 6
- tipo: HELICÓPTERO  acidentes: 5
- tipo: ULTRALEVE    acidentes: 2
aeronave mais antiga: 1946 - fabricante: CESSNA AIRCRAFT
aeronave mais nova: 2011 - quantidade de assentos: 2 - tipo de motor: PISTÃO
quantidade de acidentes de helicóptero após 1997: 2
quantidade de acidentes de avião do fabricante CESSNA AIRCRAFT: 2


<hr>

## Exercício 3

### Considere um arquivo de entrada no formato CSV (comma separated values) com informações relativas gastos de parlamentares do estado de São Paulo, entre os anos de 2009 e 2018 (cota_parlamentar_sp.csv). As informações estão separadas pelo caracter separador ; conforme o exemplo abaixo:

```javascript
01/01/2018 00:00;3011;2015;2018;1;PPS;ALEX MANENTE;MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR;;CENTRO EMPRESARIAL MEDITERRANEO;91606
01/01/2018 00:00;3011;2015;2018;1;PPS;ALEX MANENTE;MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR;;CENTRO EMPRESARIAL MEDITERRANEO;86864
01/01/2018 00:00;3011;2015;2018;1;PPS;ALEX MANENTE;MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR;;CENTRO EMPRESARIAL MEDITERRANEO;90555
```

### As colunas estão organizadas da segunte forma:
1. datemissao
2. nudeputadoid
3. nulegislatura
4. numano
5. nummes
6. sgpartido
7. txnomeparlamentar
8. txtdescricao
9. txtdescricaoespecificacao 
10. txtfornecedor 
11. vlrdocumento

### Crie uma função que efetue a leitura do arquivo (cota_parlamentar_sp.csv) e para cada legenda (sgpartido) grave um arquivo com o nome da legenda no formato JSON (ex: DEM.json, PSDB.json)

### Em cada arquivo deverá conter as seguintes informações conforme exemplo:

```json
[
  {
    "legenda": "DEM",
    "nome": "ROBSON TUMA",
    "total": 177565
  },
  {
    "legenda": "DEM",
    "nome": "JORGINHO MALULY",
    "total": 31335515
  },
  {
    "legenda": "SD",
    "nome": "PAULO PEREIRA DA SILVA", 
    "total": 155445892
  }
]
```

* legenda (partido)
* nome (nome do parlamentar)
* total (total de despesas do parlamentar)

OBS: 
1. Efetuar todas as leituras e gravações de arquivos na pasta /tmp

In [7]:
import csv, json

def partidos(file):
    
    d = {}
    count = 0 # just for time-saving
    
    # open file with csv library
    with open(file, 'r', encoding="utf-8") as csv_file:
        
        csv_reader = csv.DictReader(csv_file, delimiter = ';')
        
        # iterate through source file
        for row in csv_reader:
            count += 1
            
            key1 = row['sgpartido']
            
            key2 = row['txnomeparlamentar']

            if key1 not in d:
                d[key1] = {}

            if key2 in d[key1]:
                d[key1][key2] += int(row['vlrdocumento'])
            else:
                d[key1][key2] = int(row['vlrdocumento'])
            
            # limit the number of lines to read for testing
            if count == 30000:
                break
                
    # ----------------------------
    # write JSON file        

    data = {} # new dict to store calculations and write
    data['parlamentar'] = []

    # iterate through dict 
    for key1 in d.keys():
        for key2 in d[key1].keys():
        
            data['parlamentar'].append({
                'legenda': key1,
                'nome': key2,
                'total': d[key1][key2]
            })
        
        # write to JSON file forcing indent and encoding
        with open(key1+'.json', 'w', encoding='utf-8') as outfile:
            json.dump(data['parlamentar'], outfile, ensure_ascii=False, indent=4)
            
        data['parlamentar'] = []

In [8]:
partidos('cota_parlamentar_sp.csv')

<hr>